In [1]:
!pip install statsmodels
!pip install hyperas
!pip install hyperopt

In [2]:
import os
import pandas as pd
import zipfile
import numpy as np
import tensorflow as tf
import h5py
import random
from scipy import ndimage
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn
import tempfile
import datetime
from statsmodels.stats.proportion import proportion_confint
from functions.auc_delong_xu import auc_ci_Delong

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.layers.merge import concatenate
from tensorflow.keras import regularizers
from keras import backend as K

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import Session
import gc

from keras import backend as K

In [16]:
K.clear_session()
reset_keras()

169


In [17]:
!nvidia-smi

Wed Apr 21 18:38:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:05:00.0 Off |                    0 |
| N/A   47C    P0    54W / 250W |  21793MiB / 22919MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
 def reset_keras():
        """
        Resets the Tensorflow graph and clears the session. Should be executed after eevery traning session with a model.

        :return:
        """
        sess = tf.compat.v1.keras.backend.get_session()
        tf.keras.backend.clear_session()
        sess.close()
        sess = tf.compat.v1.keras.backend.get_session()

        try:
            del model # this is from global space - change this as you need
        except:
            pass

        print(gc.collect()) # if it's done something you should see a number being outputted

        # use the same config as you used to create the session
        config = ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 0.5
        config.gpu_options.visible_device_list = "0"
        tf.compat.v1.keras.backend.set_session(Session(config=config))

#### Import data

In [7]:
def data():
    ##Import data from hd5f file
    PATH_3D_H5 = 'Images/preprocessed_data_64_56_20.hdf5'
    with h5py.File(PATH_3D_H5, 'r') as h5:
        print('H5-file: ', list(h5.keys()))

         # Training data
        X_train = h5["X_train"][:]
        Y_train = h5["Y_train"][:]
        pat_train = h5["pat_train"][:]

        # Test
        X_test = h5["X_test"][:]
        Y_test = h5["Y_test"][:]
        pat_test = h5["pat_test"][:]

    print(X_train.shape, X_train.min(), X_train.max(), X_train.mean(), X_train.std(), Y_train.shape)
    print(X_test.shape, X_test.min(), X_test.max(), X_test.mean(), X_test.std(), Y_test.shape)
    
    ###Create balanced dataset
    bool_train_labels = Y_train[:,1] != 0
    pos_features = X_train[bool_train_labels]
    neg_features = X_train[~bool_train_labels]
    pos_labels = Y_train[bool_train_labels]
    neg_labels = Y_train[~bool_train_labels]

    ids = np.arange(len(neg_features))
    choices = np.random.choice(ids, len(pos_features))
    res_neg_features = neg_features[choices]
    res_neg_labels = neg_labels[choices]

    resampled_features = np.concatenate([res_neg_features, pos_features], axis=0)
    resampled_labels = np.concatenate([res_neg_labels, pos_labels], axis=0)

    order = np.arange(len(resampled_labels))
    np.random.shuffle(order)
    X_train_balanced = resampled_features[order]
    Y_train_balanced = resampled_labels[order]

    print(X_train_balanced.shape, Y_train_balanced.shape)
    
    return X_train_balanced, Y_train_balanced, X_test, Y_test

In [18]:
def model(X_train, Y_train, X_valid, Y_valid):
    # Architecture: 
    # - dropout probability [0, 1]
    # - number of kernels {32, 64, 128}
    # - kernel size {3, 5, 7}
    # 
    # Optimization:
    # - learning rate [0.001, 0.1]
    # - batch size {64, 128, 256}
    
    def reset_keras():
        """
        Resets the Tensorflow graph and clears the session. Should be executed after eevery traning session with a model.

        :return:
        """
        sess = tf.compat.v1.keras.backend.get_session()
        tf.keras.backend.clear_session()
        sess.close()
        sess = tf.compat.v1.keras.backend.get_session()

        try:
            del model # this is from global space - change this as you need
        except:
            pass

        print(gc.collect()) # if it's done something you should see a number being outputted

        # use the same config as you used to create the session
        config = ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 1
        config.gpu_options.visible_device_list = "0"
        tf.compat.v1.keras.backend.set_session(Session(config=config))
        
    reset_keras()

    num_classes = 2
    model = keras.Sequential()
    model.add(layers.Convolution3D({{choice([8, 16, 32])}}, kernel_size=3, activation="relu", 
                                   batch_input_shape=(None,64,56,20,1), padding = 'same'))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([16, 32, 64])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([16, 32, 64])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([16, 32, 64])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([32, 64, 128])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    model.add(layers.MaxPooling3D(pool_size = 2, padding = 'same'))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Flatten())
    model.add(layers.Dense({{choice([32, 64, 128])}}, activation="relu"))
    model.add(layers.Dropout({{uniform(0, 0.5)}}))
    
    model.add(layers.Dense(num_classes, activation='softmax'))
              
    #compile model
    metrics = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')]

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
                  metrics=metrics)
    
    history = model.fit(X_train_balanced, Y_train_balanced,
                        validation_data=(X_test, Y_test), 
                        batch_size={{choice([2,4,6])}},
                        verbose=2,
                        epochs=50)
    
    #get the highest validation accuracy of the training epochs
    val_auc = np.amax(history.history['val_auc']) 
    print('Best validation auc of epoch:', val_auc)
              
    return {'loss': -val_auc, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      max_evals=50,
                                      algo=tpe.suggest,
                                      notebook_name="CNN_hyperparameter_tuning", # Without this it can't find the notebook!
                                      trials=Trials())

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import zipfile
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import h5py
except:
    pass

try:
    import random
except:
    pass

try:
    from scipy import ndimage
except:
    pass

try:
    import shutil
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.metrics import confusion_matrix
except:
    pass

try:
    import sklearn
except:
    pass

try:
    import tempfile
except:
    pass

try:
    import datetime
except:
    pass

try:
    from statsmodels.stats.proportion import proportion_confint
except:
    pass

try:
    from functions.auc_delong_xu import auc_ci_Delong
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from tensorflow.keras import layers
except:
    pass

try:
    from keras.utils import 

In [10]:
best_run

NameError: name 'best_run' is not defined

In [55]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 64, 56, 20, 16)    448       
_________________________________________________________________
dropout (Dropout)            (None, 64, 56, 20, 16)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 32, 28, 10, 16)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 28, 10, 16)    64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 28, 10, 32)    13856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 28, 10, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 16, 14, 5, 32)     0